# Construction d'un embedding

In [2]:
import pandas as pd
from gensim.test.utils import common_texts
from gensim.models import FastText, KeyedVectors
import numpy as np
import gensim
import sys
import clean_text


import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin


In [11]:
df_result = pd.DataFrame(data = {'pipeline 1' : [0,0,0],'pipeline 2' : [0,0,0],'pipeline 3' : [0,0,0]},index=['accuracy','ballanced accuracy','f1-weighted']).T

In [12]:
df_result

,accuracy,ballanced accuracy,f1-weighted
pipeline 1,0,0,0
pipeline 2,0,0,0
pipeline 3,0,0,0


In [22]:
from pprint import pprint
from time import time
import logging

import pandas as pd
import gensim
import numpy as np
import sklearn as sk
import seaborn as sns

import nltk
from nltk import word_tokenize
lang ='french'

import clean_text



from scipy.stats import randint
from scipy.sparse import csr_matrix


import matplotlib.pyplot as plt


from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import TruncatedSVD,IncrementalPCA,SparsePCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer

import spacy
nlp =spacy.load('fr')
from spacy.lang.fr.stop_words import STOP_WORDS

In [5]:
df_declaration_mrv = pd.read_csv("data/data_mrv/declaration_mrv.csv",delimiter=';',encoding='ISO-8859-1')
id_to_dco = pd.read_csv("data/ref_MRV/referentiel_dispositif.csv",delimiter=';',encoding='ISO-8859-1')

df = df_declaration_mrv[['DESCRIPTION_INCIDENT','LIBELLE_COMMERCIAL','DCO_ID']]


df['Text'] = df['LIBELLE_COMMERCIAL']+ ' ' + df['DESCRIPTION_INCIDENT']

df = df.dropna()

/home/user1/DGS-env/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [8]:
%%time
df.Text = df.Text.map(lambda x: clean_text.preprocess_text(x))

CPU times: user 9.83 s, sys: 72 ms, total: 9.9 s
Wall time: 9.91 s


In [9]:
SENTENCES = df.Text.to_list()

In [10]:
%%time
model_dco = FastText(sentences=SENTENCES,size=300,window=5,negative=10,sg=0,min_n=5,max_n=4,workers=32) 
model_dco.save('emb_Text.bin')

CPU times: user 13min 24s, sys: 27.2 s, total: 13min 51s
Wall time: 2min 25s


In [41]:
from sklearn.base import BaseEstimator, TransformerMixin


class  ReshapeVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = model
        self.dim = 768

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.model[w] for w in words if w in self.model]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [23]:
def select_raw_by_nb_obs(df:pd.DataFrame, seuil:int)->pd.DataFrame :
    """
    Renvoie les lignes ou le nombre d'observations est supérieur au seuil entrée
    """
    S = df.groupby('DCO_ID').count()>seuil
    liste_DCO =S[S['Text']==True].index
    df_utilisable= df[df['DCO_ID'].isin(liste_DCO)]
    #df_reduit = df_utilisale[df_utilisale['DCO_ID']>2900]
    #print(len(df_reduit))
    return(df_utilisable)

df_10 = select_raw_by_nb_obs(df,10)
df_10_unique = df_10.drop_duplicates('DESCRIPTION_INCIDENT') 

In [43]:
%%time
embeddings_pipeline = Pipeline(
    steps=[
        ("embeddings", FasttextVectorTransformer(model_dco)),
        ("classifier", LinearSVC(class_weight='balanced')),
    ]
)
X_train_u, X_test_u, y_train_u, y_test_u = train_test_split(df_10_unique['LIBELLE_COMMERCIAL'],df_10_unique.DCO_ID,test_size=0.25)
embeddings_pipeline.fit(X_train_u, y_train_u)
y_pred_u = embeddings_pipeline.predict(X_test_u)

print("précison:", accuracy_score(y_test_u,y_pred_u) )
print("Balanced_accuracy : ", balanced_accuracy_score(y_test_u,y_pred_u))

/home/user1/DGS-env/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()
/home/user1/DGS-env/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/user1/DGS-env/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()
/home/user1/DGS-env/l

précison: 0.22256449749711205
Balanced_accuracy :  0.15640627051102393
CPU times: user 1h 15s, sys: 15.4 s, total: 1h 30s
Wall time: 1h 38s


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
